In [1]:
from __future__ import print_function, division, absolute_import

# python imports
from copy import copy
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/Downloads/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [2]:
eco_directory = join(flat_files.ecoli_files_dir, 'iJO1366.json')
ijo_directory = join(flat_files.ecoli_files_dir, 'iYO844.json')
uni_directory = join(flat_files.ecoli_files_dir, 'universal_model.json')

eco = cobra.io.load_json_model(eco_directory)
bsub = cobra.io.load_json_model(ijo_directory)
uni = cobra.io.load_json_model(uni_directory)

bsub.optimize()

<Solution 0.12 at 0x7fb55ae6f940>

In [3]:
with open('./me_models/iBS.pickle', 'rb') as f:
    me = pickle.load(f)

In [4]:
solve_me_model(me, 1., min_mu = .1, precision=1e-2, using_soplex=False)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 94.893968 seconds
Finished substituting S,lb,ub in 3.381221 seconds
Finished makeME_LP in 0.654457 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.5 0.5 1
Finished substituting S,lb,ub in 3.334421 seconds
Finished makeME_LP in 0.651895 seconds
Getting MINOS parameters from ME_NLP...
2 0.0 0.0 0.25 0.25 1
Finished substituting S,lb,ub in 3.330354 seconds
Finished makeME_LP in 0.676111 seconds
Getting MINOS parameters from ME_NLP...
3 0.0 0.0 0.125 0.125 1
Finished substituting S,lb,ub in 3.374699 seconds
Finished makeME_LP in 0.675227 seconds
Getting MINOS parameters from ME_NLP...
4 0.0625 0.0625 0.125 0.0625 optimal
Finished substituting S,lb,ub in 3.425889 seconds
Finished makeME_LP in 0.645596 seconds
Getting MINOS parameters from ME_NLP...
5 0.09375 0.09375 0.125 0.09375 optimal
Finished substituting S,lb,ub in 3.320446 seconds
Finished makeME_LP in 0.660151 seconds
Getting MINOS parameters 

In [5]:
with open('./me_models/solution.pickle', 'wb') as solution:
    pickle.dump(me,solution)

In [6]:
with open('./me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

In [7]:
df_m = exchange_single_model(bsub)
df_m

,id,name,reaction,lower_bound,upper_bound,flux
0,EX_ca2_e,Calcium exchange,ca2_e <=>,-999999.0,999999.0,-0.000378
1,EX_co2_e,CO2 exchange,co2_e <=>,-999999.0,999999.0,5.980676
2,EX_fe3_e,Fe3+ exchange,fe3_e <=>,-999999.0,999999.0,-0.000407
3,EX_glc__D_e,D-Glucose exchange,glc__D_e <=>,-1.7,999999.0,-1.700000
4,EX_h2o_e,H2O exchange,h2o_e <=>,-999999.0,999999.0,7.869819
5,EX_h_e,H+ exchange,h_e <=>,-999999.0,999999.0,0.767665
6,EX_k_e,K+ exchange,k_e <=>,-999999.0,999999.0,-0.083321
7,EX_mg2_e,Mg exchange,mg2_e <=>,-999999.0,999999.0,-0.011999
8,EX_nh4_e,Ammonia exchange,nh4_e <=>,-5.0,999999.0,-0.944964
9,EX_o2_e,O2 exchange,o2_e <--,-999999.0,0.0,-5.705899


In [8]:
df = exchange_single_model(me)
df

,id,name,reaction,lower_bound,upper_bound,flux
0,EX_ca2_e,,ca2_e <=>,-999999.0,999999.0,-1.766083e-28
1,EX_co2_e,,co2_e <=>,-999999.0,999999.0,6.488141e+00
2,EX_glc__D_e,,glc__D_e <=>,-1.7,999999.0,-1.700000e+00
3,EX_glyb_e,,glyb_e -->,0.0,999999.0,4.577704e-22
4,EX_h2o_e,,h2o_e <=>,-999999.0,999999.0,8.330686e+00
5,EX_h_e,,h_e <=>,-999999.0,999999.0,2.627595e-01
6,EX_hexs_e,,hexs_e -->,0.0,999999.0,-2.095133e-23
7,EX_hqn_e,,hqn_e -->,0.0,999999.0,1.219146e-34
8,EX_k_e,,k_e <=>,-999999.0,999999.0,-1.207822e-07
9,EX_mg2_e,,mg2_e <=>,-999999.0,999999.0,-3.268522e-04


In [9]:
filename = 'solution_summary.csv'
file = open(filename,'w')
file.write('ID, reaction, LB, UB, V \n')
for rxn in me.reactions:
    try:
        reaction = rxn.reaction
    except:
        reaction = 'NA'
    string = rxn.id + ',' + reaction + ',' + str(rxn.lower_bound) + ',' + str(rxn.upper_bound) +\
    ',' +  str(me.solution.x_dict[rxn.id]) + '\n'
    file.write(string)
file.close()

In [10]:
flux_dict = me.get_metabolic_flux()
filename = 'fluxdist_me_bsub.csv'
file = open(filename,'w')
for rxn in flux_dict.keys():
    flux = flux_dict[rxn]
    file.write(rxn + ',' + str(flux) + '\n')
file.close()

In [11]:
filename = 'fluxdist_m_bsub.csv'
file = open(filename,'w')
for rxn in bsub.reactions:
    flux = bsub.solution.x_dict[rxn.id]
    file.write(rxn.id + ',' + str(flux) + '\n')
file.close()

## Postprocessing

In [12]:
from __future__ import print_function, division, absolute_import

# python imports
import re
from os.path import join
from collections import defaultdict
import pickle

# third party imports
import pandas
import cobra

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import ecolime
from ecolime import (transcription, translation, flat_files, generics, formulas, compartments)
from ecolime.util.helper_functions import *

print(cobrame.__file__)
print(ecolime.__file__)

/home/jt/UCSD/cobrame/cobrame/__init__.py
/home/jt/UCSD/bacillusme-master/ecolime/__init__.py


In [13]:
with open('./me_models/solution.pickle', 'rb') as solution:
    me = pickle.load(solution)

In [14]:
me.solution.status

'optimal'

In [21]:
flux_based_reactions(me,'amp_c',-1,['translation','transcription','formation'],0.2)

ADK1_FWD_BSU01370-MONOMER_mod_mg2 ( -0.38999094393715655 ) 4.27350427350427e-6*mu BSU01370-MONOMER_mod_mg2 + amp_c + atp_c --> 2.0 adp_c
NTD7_FWD_BSU07840-MONOMER ( 8.566624133528062e-23 ) 4.27350427350427e-6*mu BSU07840-MONOMER + amp_c + h2o_c --> adn_c + pi_c


In [ ]:
me.metabolites.get_by_id('cobalt2_c').reactions

In [ ]:
me.metabolites.get_by_id('BSU32100-MONOMER').formula_weight

In [6]:
me.genes = bsub.genes
GE_dict = gene_essentiality(bsub, model_type = 'm',  lim = 0.01, NP = 20)

Number of processors:  20


In [7]:
GE_dict

{}